In [ ]:
import numpy as np, pandas as pd, os
import matplotlib.pyplot as plt, cv2
import tensorflow as tf, re, math
from tqdm import tqdm 

In [ ]:


# PATHS TO IMAGES
PATH = '../input/shopee-product-matching/train_images/'
PATH2 ='../input/shopee-product-matching/test_images/'
IMGS = os.listdir(PATH); 
IMGS2 = os.listdir(PATH2)
print('There are %i train images and %i test images'%(len(IMGS),len(IMGS2)))



In [ ]:
df = pd.read_csv('../input/shopee-product-matching/train.csv')
#df.rename({'image_id':'image_name'},axis=1,inplace=True)
df.head()
df.info()

In [ ]:
# LOAD TEST META DATA
test = pd.read_csv('../input/shopee-product-matching/test.csv')
test.head()

In [ ]:
# COMBINE TRAIN AND TEST TO ENCODE TOGETHER
cols = test.columns
comb = pd.concat([df[cols],test[cols]],ignore_index=True,axis=0).reset_index(drop=True)

In [ ]:
# LABEL ENCODE ALL STRINGS
cats = ['posting_id','image_phash','title',] 
for c in cats:
    comb[c],mp = comb[c].factorize()
    print(mp)


In [ ]:
# REWRITE DATA TO DATAFRAMES
df[cols] = comb.loc[:df.shape[0]-1,cols].values
test[cols] = comb.loc[df.shape[0]:,cols].values

In [ ]:
df

In [ ]:
import numpy as np, pandas as pd, os
import matplotlib.pyplot as plt, cv2
import tensorflow as tf, re, math
from tqdm import tqdm

In [ ]:


def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
        return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


In [ ]:
def serialize_example(feature0, feature1, feature2, feature3, feature4, feature5):
    feature = {
        'posting_id': _bytes_feature(feature0),
        'image': _bytes_feature(feature1),
        'image_name': _bytes_feature(feature2),
        'image_phash': _bytes_feature(feature3),
        'title': _bytes_feature(feature4),
        'label_group': _int64_feature(feature5)
    }

In [ ]:
import numpy as np, pandas as pd, os
import matplotlib.pyplot as plt, cv2
import tensorflow as tf, re, math
from tqdm import tqdm

#df = pd.read_csv('../input/shopee-product-matching/train.csv')
#df.head(2)
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _int64_feature(value):
        return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(feature0, feature1, feature2, feature3, feature4, feature5):
    feature = {
    'posting_id': _bytes_feature(feature0),
    'image': _bytes_feature(feature1),
    'image_name': _bytes_feature(feature2),
    'image_phash': _bytes_feature(feature3),
    'title': _bytes_feature(feature4),
    'label_group': _int32_feature(feature5)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

PATH = '../input/shopee-product-matching/train_images/'
IMGS = list(df.image)

SIZE = 2071
CT = len(IMGS)//SIZE + int(len(IMGS)%SIZE!=0)
for j in tqdm(range(CT)):
    CT2 = min(SIZE,len(IMGS)-j*SIZE) 
    with tf.io.TFRecordWriter('train%.2i-%i.tfrec'%(j,CT2)) as writer: 
        for k in range(CT2): img = cv2.imread(PATH+IMGS[SIZE*j+k])
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # Fix incorrect colors
        img = cv2.imencode('.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, 94))[1].tobytes()
        name = IMGS[SIZE*j+k]
        row = df.loc[df.image==name]
        example = serialize_example(
                row.posting_id.values[0],
                img,
                name,
                row.image_phash.values[0],
                row.title.values[0],
                row.label_group.values[0])
        writer.write(example)

In [ ]:
df